Net1


In [6]:
import torch
import torchvision
import torchaudio
import librosa
print("Everything loaded fine ✅")

Everything loaded fine ✅


In [7]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((180,180)), # Resize to 180x180 pixels
        transforms.ToTensor() # Convert to tensor
    ])

    dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    # Split the dataset into training, validation and test sets (70% train, 20% val, 10% test)
    total_length = len(dataset)
    train_length = int(0.7 * total_length)
    val_length = int(0.2 * total_length)
    test_length = total_length - train_length - val_length
    train_set, val_set, test_set = random_split(dataset, [train_length, val_length, test_length])

    # Create data loaders for each set
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [14]:
data_path = "./Data/images_original"

train_loader, val_loader, test_loader = get_dataloaders(data_path)

for images, labels in train_loader:
    print("Batch of images shape:", images.shape)  # (batch_size, channels, height, width)
    print("Batch of labels shape:", labels.shape)  # (batch_size,)
    print("image dtype:", images.dtype)  # Should be torch.float32
    break  # Just show the first batch

Batch of images shape: torch.Size([32, 3, 180, 180])
Batch of labels shape: torch.Size([32])
image dtype: torch.float32


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(3*180*180, 512)  # Input layer
        self.fc2 = nn.Linear(512, 128)  # Hidden layer
        self.output = nn.Linear(128, 10)  # Output layer (10 classes)

    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the input
        x = F.relu(self.fc1(x)) # Activation function for the first layer
        x = F.relu(self.fc2(x)) # Activation function for the second layer
        x = self.output(x)
        return x

In [16]:
model = Net1()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
loss_fn = nn.CrossEntropyLoss()

In [17]:
def train(model, train_loader, val_loader, optimizer, loss_fn, epochs=50):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

In [18]:
train (model, train_loader, val_loader, optimizer, loss_fn, epochs=50)
torch.save(model.state_dict(), "net1_fully_connected.pth")

Epoch 1/50, Loss: 208.0826
Epoch 2/50, Loss: 54.0659
Epoch 3/50, Loss: 45.8908
Epoch 4/50, Loss: 45.2979
Epoch 5/50, Loss: 42.0981
Epoch 6/50, Loss: 42.7955
Epoch 7/50, Loss: 38.3343
Epoch 8/50, Loss: 38.2002
Epoch 9/50, Loss: 37.5623
Epoch 10/50, Loss: 33.9468
Epoch 11/50, Loss: 35.1454
Epoch 12/50, Loss: 33.9222
Epoch 13/50, Loss: 33.0339
Epoch 14/50, Loss: 30.2270
Epoch 15/50, Loss: 28.5967
Epoch 16/50, Loss: 29.2423
Epoch 17/50, Loss: 26.2354
Epoch 18/50, Loss: 25.5730
Epoch 19/50, Loss: 24.0937
Epoch 20/50, Loss: 23.6947
Epoch 21/50, Loss: 22.7962
Epoch 22/50, Loss: 20.0588
Epoch 23/50, Loss: 18.1404
Epoch 24/50, Loss: 17.2349
Epoch 25/50, Loss: 21.2168
Epoch 26/50, Loss: 20.1062
Epoch 27/50, Loss: 18.9440
Epoch 28/50, Loss: 24.6866
Epoch 29/50, Loss: 16.1873
Epoch 30/50, Loss: 15.0658
Epoch 31/50, Loss: 10.7541
Epoch 32/50, Loss: 10.1577
Epoch 33/50, Loss: 9.1291
Epoch 34/50, Loss: 9.0246
Epoch 35/50, Loss: 12.4966
Epoch 36/50, Loss: 12.0020
Epoch 37/50, Loss: 9.4585
Epoch 38/50,

In [19]:
def evaluate_accuracy(model, data_loader):
    model.eval()
    total_correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_correct / total

In [ ]:
train_acc = evaluate_accuracy(model, train_loader)
val_acc = evaluate_accuracy(model, val_loader)
test_acc = evaluate_accuracy(model, test_loader)

print(f"✅ Train Acc: {train_acc:.2%}")
print(f"✅ Val Acc: {val_acc:.2%}")
print(f"✅ Test Acc: {test_acc:.2%}")

Net2


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # [B, 3, 180, 180] → [B, 16, 180, 180]
        self.pool = nn.MaxPool2d(2, 2)                           # → [B, 16, 90, 90]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # → [B, 32, 90, 90] → pool → [B, 32, 45, 45]

        self.fc1 = nn.Linear(32 * 45 * 45, 256)
        self.fc2 = nn.Linear(256, 64)
        self.out = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + MaxPool
        x = x.view(x.size(0), -1)             # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)                       # Output logits
        return x


In [12]:
model = Net2()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [13]:
train(model, train_loader, val_loader, optimizer, loss_fn, epochs=50)
torch.save(model.state_dict(), "net2_cnn.pth")

Epoch 1/50, Loss: 51.6608
Epoch 2/50, Loss: 47.4494
Epoch 3/50, Loss: 44.2544
Epoch 4/50, Loss: 41.3189
Epoch 5/50, Loss: 37.0602
Epoch 6/50, Loss: 33.2790
Epoch 7/50, Loss: 31.1709
Epoch 8/50, Loss: 28.0125
Epoch 9/50, Loss: 25.6820
Epoch 10/50, Loss: 22.9186
Epoch 11/50, Loss: 19.2013
Epoch 12/50, Loss: 18.6537
Epoch 13/50, Loss: 14.3975
Epoch 14/50, Loss: 11.3282
Epoch 15/50, Loss: 9.5058
Epoch 16/50, Loss: 7.1694
Epoch 17/50, Loss: 4.7678
Epoch 18/50, Loss: 4.5256
Epoch 19/50, Loss: 3.1985
Epoch 20/50, Loss: 2.0553
Epoch 21/50, Loss: 1.4880
Epoch 22/50, Loss: 1.2017
Epoch 23/50, Loss: 1.0376
Epoch 24/50, Loss: 0.8014
Epoch 25/50, Loss: 0.8899
Epoch 26/50, Loss: 0.7571
Epoch 27/50, Loss: 0.4422
Epoch 28/50, Loss: 0.3731
Epoch 29/50, Loss: 0.4156
Epoch 30/50, Loss: 0.2498
Epoch 31/50, Loss: 0.1587
Epoch 32/50, Loss: 0.2204
Epoch 33/50, Loss: 0.6505
Epoch 34/50, Loss: 0.3357
Epoch 35/50, Loss: 0.2208
Epoch 36/50, Loss: 0.4677
Epoch 37/50, Loss: 0.3445
Epoch 38/50, Loss: 0.3351
Epoch 3

In [ ]:
train_acc = evaluate_accuracy(model, train_loader)
val_acc = evaluate_accuracy(model, val_loader)
test_acc = evaluate_accuracy(model, test_loader)

print(f"✅ Train Acc: {train_acc:.2%}")
print(f"✅ Val Acc: {val_acc:.2%}")
print(f"✅ Test Acc: {test_acc:.2%}")

Net3


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net3(nn.Module):  # CNN with 4 convs and batchnorm
    def __init__(self):
        super(Net3, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.pool1 = nn.MaxPool2d(2, 2)  # After conv2

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.pool2 = nn.MaxPool2d(2, 2)  # After conv4

        # Output shape after pool2 = [B, 128, 45, 45] → flatten
        self.fc1 = nn.Linear(128 * 45 * 45, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [21]:
model = Net3()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train(model, train_loader, val_loader, optimizer, loss_fn, epochs=50)

Epoch 1/50, Loss: 1248.8341


KeyboardInterrupt: 